# Completitud, Variabilidad y Estabilidad de variables explicativas

In [67]:
import pandas as pd 
import numpy as np 
import datetime
from dateutil import relativedelta
import s3fs
import pyarrow.parquet as pq
fs = s3fs.S3FileSystem()

pd.set_option('max_columns',None)
pd.set_option('max_rows',100)

In [5]:
#Lectura bases 
input = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/total_tdc_paprob'
dataset = pq.ParquetDataset(input, filesystem=fs)
table = dataset.read()
base_preaprob = table.to_pandas()

del input, dataset, table


In [3]:
# input = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/total_tdc_perf'
# dataset = pq.ParquetDataset(input, filesystem=fs)
# table = dataset.read()
# base_perfilados = table.to_pandas()

# del input, dataset, table


In [6]:
base_preaprob['fecha_psi']=base_preaprob['data_camp'].astype('str')
# base_perfilados['fecha_psi']=base_perfilados['data_camp'].astype('str')

#### Asignacion de Base

In [7]:
master_table=base_preaprob.copy()
# master_table=base_perfilados.copy()

### 1.Completitud

In [8]:
completitud=pd.DataFrame(master_table.isnull().sum()/master_table.shape[0],columns = ['Porcentaje_Missing']).reset_index()

### 2. Variabilidad

In [9]:
#Se calcula el p10 y p90 para las variables
p10= pd.DataFrame(master_table.quantile(.1).reset_index())
p90= pd.DataFrame(master_table.quantile(.9).reset_index())

In [13]:
variabilidad=p10.merge(p90,how='left',on='index')
variabilidad['Percentil10']=variabilidad.iloc[:,1]
variabilidad['Percentil90']=variabilidad.iloc[:,2]

In [16]:
variabilidad['variabilidad']=np.where(variabilidad['Percentil10']==variabilidad['Percentil90'],'No variable',
                             np.where((np.isnan(variabilidad['Percentil10'])) & (np.isnan(variabilidad['Percentil90'])),'No variable','Variable'))

In [17]:
variabilidad=variabilidad[['index','Percentil10','Percentil90','variabilidad']]  

In [18]:
variabilidad

,index,Percentil10,Percentil90,variabilidad
0,id_cliente,1.915050e+17,9.115247e+17,Variable
1,fecha_tx_pav,2.019020e+05,2.019100e+05,Variable
2,fecha_tx_act,2.019020e+05,2.019100e+05,Variable
3,fecha_pasivo,2.019010e+05,2.019090e+05,Variable
4,fecha_lib,2.019020e+05,2.019100e+05,Variable
...,...,...,...,...
1488,prom_mov_ult_4meses_ahr,NaN,NaN,No variable
1489,prom_mov_ult_6meses_ahr,NaN,NaN,No variable
1490,sum_tx_linea_ahr,1.000000e+00,1.800000e+01,Variable
1491,sum_tx_nolinea_ahr,1.000000e+00,3.200000e+01,Variable


### 3. Estabilidad

In [19]:
#Función que calcula el indice de estabilidad poblacional PSI para las variables continuas
def psi_continuas(variable,base,var_fecha):
    base['decil_'+variable]=pd.qcut(base[variable],5,duplicates='drop')
    tabla_5=base.groupby(['decil_'+variable])['decil_'+variable].count().reset_index(name='conteo')
    variable_total=tabla_5['conteo'].sum()
    tabla_5['dist']=tabla_5['conteo']/variable_total

    tabla_6=base.groupby(['decil_'+variable,var_fecha])['decil_'+variable].count().reset_index(name='conteo')
    tabla_7=base.groupby([var_fecha])['decil_'+variable].count().reset_index(name='total')

    tabla_8=tabla_6.merge(tabla_7,how='left',on=[var_fecha])
    tabla_8['dist_mes']=tabla_8['conteo']/tabla_8['total']

    tabla_9=tabla_8.merge(tabla_5,how='left',on=['decil_'+variable]).sort_values(by=[var_fecha]).reset_index()
    tabla_9['PSI']=(tabla_9['dist_mes']-tabla_9['dist'])*np.log(tabla_9['dist_mes']/tabla_9['dist'])

    tabla_10=tabla_9.groupby([var_fecha])['PSI'].sum().reset_index(name='decil_'+variable)
    return pd.DataFrame([[variable,tabla_10.max()[1]]])


In [21]:
#Calculo de minimo y maximo por variale
min_=pd.DataFrame(master_table.min().reset_index())
max_=pd.DataFrame(master_table.max().reset_index())

In [24]:
min_.merge(max_,how='left',on='index').to_csv("/mnt/work/CU_adquisicionTC/Notebooks/creacion_modelo/min_max_preaprob.csv")

In [14]:
# min_.merge(max_,how='left',on='index').to_csv("/mnt/work/CU_adquisicionTC/Notebooks/creacion_modelo/min_max_perf.csv")

In [73]:
#Listado de variables continuas,no se deben incluir variables que estan todo en 0 o vacio, revisar min_ y max_ celda anterior
variables_continuas_pre=['acep_oferta_prev',
'acierta_a_financiero',
'amortizacion_max_CE',
'amortizacion_max_CH',
'amortizacion_max_LB',
'amortizacion_max_LE',
'amortizacion_max_NV',
'amortizacion_max_tot',
'amortizacion_min_CE',
'amortizacion_min_CH',
'amortizacion_min_LB',
'amortizacion_min_LE',
'amortizacion_min_NV',
'amortizacion_min_tot',
'amortizacion_prom_CE',
'amortizacion_prom_CE_ult3_meses',
'amortizacion_prom_CE_ult9_meses',
'amortizacion_prom_CH',
'amortizacion_prom_CH_ult3_meses',
'amortizacion_prom_CH_ult9_meses',
'amortizacion_prom_LB',
'amortizacion_prom_LB_ult3_meses',
'amortizacion_prom_LB_ult9_meses',
'amortizacion_prom_LE',
'amortizacion_prom_LE_ult3_meses',
'amortizacion_prom_LE_ult9_meses',
'amortizacion_prom_NV',
'amortizacion_prom_NV_ult3_meses',
'amortizacion_prom_NV_ult9_meses',
'amortizacion_prom_tot',
'amortizacion_prom_tot_ult3_meses',
'amortizacion_prom_tot_ult9_meses',
'ano_po',
'antiguedad_TDC_T',
'antiguedad_TDC_X',
'antiguedad_tot_conlibranza',
'antiguedad_tot_sinlibranza',
'cartera_banca_alt_max_de_mora',
'cartera_banca_alt_max_de_mora_lag1',
'cartera_castig_cartera_actual',
'cartera_castig_cartera_actualdif',
'cartera_coope_alt_max_de_mora',
'cartera_coope_alt_max_de_mora_lag1',
'cartera_hipote_alt_max_de_mora',
'cartera_hipote_alt_max_de_mora_lag1',
'cartera_recupe_cartera_hist',
'cartera_recupe_cartera_histdif',
'ctas_ctes_act_cta_banca',
'ctas_ctes_act_cta_bancadif',
'ctas_de_ahorro_act_ctas_banca',
'ctas_de_ahorro_act_ctas_bancadif',
'ctas_embargadas_ctas_bancarias',
'ctas_embargadas_ctas_bancariasdif',
'ctas_saldadas_ctas_banca',
'ctas_saldadas_ctas_bancadif',
'cuota_max_CE_SMLV',
'cuota_max_CH_SMLV',
'cuota_max_LB_SMLV',
'cuota_max_LE_SMLV',
'cuota_max_NV_SMLV',
'cuota_max_TC_SMLV',
'cuota_max_TC_T_SMLV',
'cuota_max_TC_X_SMLV',
'cuota_max_tot_SMLV',
'cuota_mercado_smlv',
'cuota_mercadodif',
'cuota_paga_prom_ult3_meses_total',
'cuota_paga_prom_ult7_meses_total',
'cuota_prom_CE_SMLV',
'cuota_prom_CH_SMLV',
'cuota_prom_LB_SMLV',
'cuota_prom_LE_SMLV',
'cuota_prom_NV_SMLV',
'cuota_prom_TC_SMLV',
'cuota_prom_TC_T_SMLV',
'cuota_prom_TC_X_SMLV',
'cuota_prom_tot_SMLV',
'cuota_tot_CE_SMLV',
'cuota_tot_CE_SMLV_ult3_meses',
'cuota_tot_CE_SMLV_ult8_meses',
'cuota_tot_CH_SMLV',
'cuota_tot_CH_SMLV_ult3_meses',
'cuota_tot_CH_SMLV_ult8_meses',
'cuota_tot_LB_SMLV',
'cuota_tot_LB_SMLV_ult3_meses',
'cuota_tot_LB_SMLV_ult8_meses',
'cuota_tot_LE_SMLV',
'cuota_tot_LE_SMLV_ult3_meses',
'cuota_tot_LE_SMLV_ult8_meses',
'cuota_tot_NV_SMLV',
'cuota_tot_NV_SMLV_ult3_meses',
'cuota_tot_NV_SMLV_ult8_meses',
'cuota_tot_TC_SMLV',
'cuota_tot_TC_SMLV_ult3_meses',
'cuota_tot_TC_SMLV_ult8_meses',
'cuota_tot_TC_T_SMLV',
'cuota_tot_TC_T_SMLV_ult3_meses',
'cuota_tot_TC_T_SMLV_ult8_meses',
'cuota_tot_TC_X_SMLV',
'cuota_tot_TC_X_SMLV_ult3_meses',
'cuota_tot_TC_X_SMLV_ult8_meses',
'cuota_tot_tot_SMLV',
'cuota_tot_tot_SMLV_ult3_meses',
'cuota_tot_tot_SMLV_ult8_meses',
'cupo_max_CE_SMLV',
'cupo_max_CH_SMLV',
'cupo_max_LB_SMLV',
'cupo_max_LE_SMLV',
'cupo_max_NV_SMLV',
'cupo_max_TC_SMLV',
'cupo_max_TC_T_SMLV',
'cupo_max_TC_X_SMLV',
'cupo_max_tot_SMLV',
'cupo_mercado_smlv',
'cupo_mercadodif',
'cupo_prom_CE_SMLV',
'cupo_prom_CH_SMLV',
'cupo_prom_LB_SMLV',
'cupo_prom_LE_SMLV',
'cupo_prom_NV_SMLV',
'cupo_prom_TC_SMLV',
'cupo_prom_TC_T_SMLV',
'cupo_prom_TC_X_SMLV',
'cupo_prom_tot_SMLV',
'cupo_prom_ult3_meses_expr',
'cupo_prom_ult3_meses_total',
'cupo_prom_ult3_meses_trad',
'cupo_prom_ult7_meses_expr',
'cupo_prom_ult7_meses_total',
'cupo_prom_ult7_meses_trad',
'cupo_sector_hip_sin_popular_smlv',
'cupo_sector_hip_sin_populardif',
'cupo_sin_popular_smlv',
'cupo_sin_populardif',
'cupo_tot_CE_SMLV',
'cupo_tot_CE_SMLV_ult3_meses',
'cupo_tot_CE_SMLV_ult9_meses',
'cupo_tot_CH_SMLV',
'cupo_tot_CH_SMLV_ult3_meses',
'cupo_tot_CH_SMLV_ult9_meses',
'cupo_tot_LB_SMLV',
'cupo_tot_LB_SMLV_ult3_meses',
'cupo_tot_LB_SMLV_ult9_meses',
'cupo_tot_LE_SMLV',
'cupo_tot_LE_SMLV_ult3_meses',
'cupo_tot_LE_SMLV_ult9_meses',
'cupo_tot_NV_SMLV',
'cupo_tot_NV_SMLV_ult3_meses',
'cupo_tot_NV_SMLV_ult9_meses',
'cupo_tot_SO_SMLV_ult9_meses',
'cupo_tot_TC_SMLV',
'cupo_tot_TC_SMLV_ult3_meses',
'cupo_tot_TC_SMLV_ult9_meses',
'cupo_tot_TC_T_SMLV',
'cupo_tot_TC_T_SMLV_ult3_meses',
'cupo_tot_TC_T_SMLV_ult9_meses',
'cupo_tot_TC_X_SMLV',
'cupo_tot_TC_X_SMLV_ult3_meses',
'cupo_tot_TC_X_SMLV_ult9_meses',
'cupo_tot_tot_SMLV',
'cupo_tot_tot_SMLV_ult3_meses',
'cupo_tot_tot_SMLV_ult9_meses',
'cuposectorbancario_sin_popular_smlv',
'cuposectorbancario_sin_populardif',
'dias_desde_ult_tx',
'dias_desde_ult_tx_ahr',
'dias_desde_ult_tx_cdt',
'dias_desde_ult_tx_cte',
'dias_desde_ult_tx_expr',
'dias_desde_ult_tx_la14',
'dias_desde_ult_tx_pasv',
'dias_desde_ult_tx_trad',
'dif_porc_t_1_amortizacion_prom_CE',
'dif_porc_t_1_amortizacion_prom_CH',
'dif_porc_t_1_amortizacion_prom_LB',
'dif_porc_t_1_amortizacion_prom_LE',
'dif_porc_t_1_amortizacion_prom_NV',
'dif_porc_t_1_amortizacion_prom_tot',
'dif_porc_t_1_cuota_tot_CE_SMLV',
'dif_porc_t_1_cuota_tot_CH_SMLV',
'dif_porc_t_1_cuota_tot_LB_SMLV',
'dif_porc_t_1_cuota_tot_LE_SMLV',
'dif_porc_t_1_cuota_tot_NV_SMLV',
'dif_porc_t_1_cuota_tot_TC_SMLV',
'dif_porc_t_1_cuota_tot_TC_T_SMLV',
'dif_porc_t_1_cuota_tot_TC_X_SMLV',
'dif_porc_t_1_cuota_tot_tot_SMLV',
'dif_porc_t_1_cupo_tot_CE_SMLV',
'dif_porc_t_1_cupo_tot_CH_SMLV',
'dif_porc_t_1_cupo_tot_LB_SMLV',
'dif_porc_t_1_cupo_tot_NV_SMLV',
'dif_porc_t_1_cupo_tot_SO_SMLV',
'dif_porc_t_1_cupo_tot_TC_SMLV',
'dif_porc_t_1_cupo_tot_TC_T_SMLV',
'dif_porc_t_1_cupo_tot_TC_X_SMLV',
'dif_porc_t_1_cupo_tot_tot_SMLV',
'dif_porc_t_1_mora_max_tot',
'dif_porc_t_1_mora_maxCH',
'dif_porc_t_1_mora_maxLB',
'dif_porc_t_1_mora_maxLE',
'dif_porc_t_1_mora_maxTC',
'dif_porc_t_1_mora_maxTC_X',
'dif_porc_t_1_porc_obl_mora1_30_CH',
'dif_porc_t_1_porc_obl_mora1_30_LB',
'dif_porc_t_1_porc_obl_mora1_30_LE',
'dif_porc_t_1_porc_obl_mora1_30_TC',
'dif_porc_t_1_porc_obl_mora1_30_TC_X',
'dif_porc_t_1_porc_obl_mora1_30_tot',
'dif_porc_t_1_porc_obl_mora31_60_LB',
'dif_porc_t_1_porc_obl_mora31_60_tot',
'dif_porc_t_1_saldo_tot_CE_SMLV',
'dif_porc_t_1_saldo_tot_CH_SMLV',
'dif_porc_t_1_saldo_tot_LB_SMLV',
'dif_porc_t_1_saldo_tot_LE_SMLV',
'dif_porc_t_1_saldo_tot_NV_SMLV',
'dif_porc_t_1_saldo_tot_SO_SMLV',
'dif_porc_t_1_saldo_tot_TC_SMLV',
'dif_porc_t_1_saldo_tot_TC_T_SMLV',
'dif_porc_t_1_saldo_tot_TC_X_SMLV',
'dif_porc_t_1_saldo_tot_tot_SMLV',
'dif_porc_t_1_utilizacion_prom_TC',
'dif_porc_t_1_utilizacion_prom_TC_T',
'dif_porc_t_1_utilizacion_prom_TC_X',
'dif_porc_t_6_amortizacion_prom_CE',
'dif_porc_t_6_amortizacion_prom_CH',
'dif_porc_t_6_amortizacion_prom_LB',
'dif_porc_t_6_amortizacion_prom_LE',
'dif_porc_t_6_amortizacion_prom_NV',
'dif_porc_t_6_amortizacion_prom_tot',
'dif_porc_t_6_cuota_tot_CE_SMLV',
'dif_porc_t_6_cuota_tot_CH_SMLV',
'dif_porc_t_6_cuota_tot_LB_SMLV',
'dif_porc_t_6_cuota_tot_LE_SMLV',
'dif_porc_t_6_cuota_tot_NV_SMLV',
'dif_porc_t_6_cuota_tot_TC_SMLV',
'dif_porc_t_6_cuota_tot_TC_T_SMLV',
'dif_porc_t_6_cuota_tot_TC_X_SMLV',
'dif_porc_t_6_cuota_tot_tot_SMLV',
'dif_porc_t_6_cupo_tot_CE_SMLV',
'dif_porc_t_6_cupo_tot_CH_SMLV',
'dif_porc_t_6_cupo_tot_LB_SMLV',
'dif_porc_t_6_cupo_tot_LE_SMLV',
'dif_porc_t_6_cupo_tot_NV_SMLV',
'dif_porc_t_6_cupo_tot_SO_SMLV',
'dif_porc_t_6_cupo_tot_TC_SMLV',
'dif_porc_t_6_cupo_tot_TC_T_SMLV',
'dif_porc_t_6_cupo_tot_TC_X_SMLV',
'dif_porc_t_6_cupo_tot_tot_SMLV',
'dif_porc_t_6_mora_max_tot',
'dif_porc_t_6_mora_maxLB',
'dif_porc_t_6_mora_maxLE',
'dif_porc_t_6_mora_maxTC',
'dif_porc_t_6_mora_maxTC_X',
'dif_porc_t_6_porc_obl_mora1_30_LB',
'dif_porc_t_6_porc_obl_mora1_30_LE',
'dif_porc_t_6_porc_obl_mora1_30_TC',
'dif_porc_t_6_porc_obl_mora1_30_TC_X',
'dif_porc_t_6_porc_obl_mora1_30_tot',
'dif_porc_t_6_porc_obl_mora31_60_LB',
'dif_porc_t_6_porc_obl_mora31_60_tot',
'dif_porc_t_6_porc_obl_mora91_120_LB',
'dif_porc_t_6_porc_obl_mora91_120_tot',
'dif_porc_t_6_saldo_tot_CE_SMLV',
'dif_porc_t_6_saldo_tot_CH_SMLV',
'dif_porc_t_6_saldo_tot_LB_SMLV',
'dif_porc_t_6_saldo_tot_LE_SMLV',
'dif_porc_t_6_saldo_tot_NV_SMLV',
'dif_porc_t_6_saldo_tot_SO_SMLV',
'dif_porc_t_6_saldo_tot_TC_SMLV',
'dif_porc_t_6_saldo_tot_TC_T_SMLV',
'dif_porc_t_6_saldo_tot_TC_X_SMLV',
'dif_porc_t_6_saldo_tot_tot_SMLV',
'dif_porc_t_6_utilizacion_prom_TC',
'dif_porc_t_6_utilizacion_prom_TC_T',
'dif_porc_t_6_utilizacion_prom_TC_X',
'dudoso_recaudo_cartera_actual',
'dudoso_recaudo_cartera_actualdif',
'endeudamiento',
'estado_consulta',
'fecha_real',
'lag_t_1_amortizacion_prom_CE',
'lag_t_1_amortizacion_prom_CH',
'lag_t_1_amortizacion_prom_LB',
'lag_t_1_amortizacion_prom_LE',
'lag_t_1_amortizacion_prom_NV',
'lag_t_1_amortizacion_prom_tot',
'lag_t_1_cuota_tot_CE_SMLV',
'lag_t_1_cuota_tot_CH_SMLV',
'lag_t_1_cuota_tot_LB_SMLV',
'lag_t_1_cuota_tot_LE_SMLV',
'lag_t_1_cuota_tot_NV_SMLV',
'lag_t_1_cuota_tot_TC_SMLV',
'lag_t_1_cuota_tot_TC_T_SMLV',
'lag_t_1_cuota_tot_TC_X_SMLV',
'lag_t_1_cuota_tot_tot_SMLV',
'lag_t_1_cupo_tot_CE_SMLV',
'lag_t_1_cupo_tot_CH_SMLV',
'lag_t_1_cupo_tot_LB_SMLV',
'lag_t_1_cupo_tot_LE_SMLV',
'lag_t_1_cupo_tot_NV_SMLV',
'lag_t_1_cupo_tot_TC_SMLV',
'lag_t_1_cupo_tot_TC_T_SMLV',
'lag_t_1_cupo_tot_TC_X_SMLV',
'lag_t_1_cupo_tot_tot_SMLV',
'lag_t_1_mora_max_tot',
'lag_t_1_mora_maxCH',
'lag_t_1_mora_maxLB',
'lag_t_1_mora_maxLE',
'lag_t_1_mora_maxTC',
'lag_t_1_mora_maxTC_X',
'lag_t_1_porc_obl_mora1_30_CH',
'lag_t_1_porc_obl_mora1_30_LB',
'lag_t_1_porc_obl_mora1_30_LE',
'lag_t_1_porc_obl_mora1_30_TC',
'lag_t_1_porc_obl_mora1_30_TC_X',
'lag_t_1_porc_obl_mora1_30_tot',
'lag_t_1_porc_obl_mora31_60_LB',
'lag_t_1_porc_obl_mora31_60_tot',
'lag_t_1_saldo_tot_CE_SMLV',
'lag_t_1_saldo_tot_CH_SMLV',
'lag_t_1_saldo_tot_LB_SMLV',
'lag_t_1_saldo_tot_LE_SMLV',
'lag_t_1_saldo_tot_NV_SMLV',
'lag_t_1_saldo_tot_TC_SMLV',
'lag_t_1_saldo_tot_TC_T_SMLV',
'lag_t_1_saldo_tot_TC_X_SMLV',
'lag_t_1_saldo_tot_tot_SMLV',
'lag_t_1_utilizacion_prom_TC',
'lag_t_1_utilizacion_prom_TC_T',
'lag_t_1_utilizacion_prom_TC_X',
'lag_t_6_amortizacion_prom_CE',
'lag_t_6_amortizacion_prom_CH',
'lag_t_6_amortizacion_prom_LB',
'lag_t_6_amortizacion_prom_LE',
'lag_t_6_amortizacion_prom_NV',
'lag_t_6_amortizacion_prom_tot',
'lag_t_6_cuota_tot_CE_SMLV',
'lag_t_6_cuota_tot_CH_SMLV',
'lag_t_6_cuota_tot_LB_SMLV',
'lag_t_6_cuota_tot_LE_SMLV',
'lag_t_6_cuota_tot_NV_SMLV',
'lag_t_6_cuota_tot_TC_SMLV',
'lag_t_6_cuota_tot_TC_T_SMLV',
'lag_t_6_cuota_tot_TC_X_SMLV',
'lag_t_6_cuota_tot_tot_SMLV',
'lag_t_6_cupo_tot_CE_SMLV',
'lag_t_6_cupo_tot_CH_SMLV',
'lag_t_6_cupo_tot_LB_SMLV',
'lag_t_6_cupo_tot_LE_SMLV',
'lag_t_6_cupo_tot_NV_SMLV',
'lag_t_6_cupo_tot_TC_SMLV',
'lag_t_6_cupo_tot_TC_T_SMLV',
'lag_t_6_cupo_tot_TC_X_SMLV',
'lag_t_6_cupo_tot_tot_SMLV',
'lag_t_6_mora_max_tot',
'lag_t_6_mora_maxLB',
'lag_t_6_mora_maxLE',
'lag_t_6_mora_maxTC',
'lag_t_6_mora_maxTC_X',
'lag_t_6_porc_obl_mora1_30_LB',
'lag_t_6_porc_obl_mora1_30_LE',
'lag_t_6_porc_obl_mora1_30_TC',
'lag_t_6_porc_obl_mora1_30_TC_X',
'lag_t_6_porc_obl_mora1_30_tot',
'lag_t_6_porc_obl_mora31_60_LB',
'lag_t_6_porc_obl_mora31_60_tot',
'lag_t_6_porc_obl_mora91_120_LB',
'lag_t_6_porc_obl_mora91_120_tot',
'lag_t_6_saldo_tot_CE_SMLV',
'lag_t_6_saldo_tot_CH_SMLV',
'lag_t_6_saldo_tot_LB_SMLV',
'lag_t_6_saldo_tot_LE_SMLV',
'lag_t_6_saldo_tot_NV_SMLV',
'lag_t_6_saldo_tot_TC_SMLV',
'lag_t_6_saldo_tot_TC_T_SMLV',
'lag_t_6_saldo_tot_TC_X_SMLV',
'lag_t_6_saldo_tot_tot_SMLV',
'lag_t_6_utilizacion_prom_TC',
'lag_t_6_utilizacion_prom_TC_T',
'lag_t_6_utilizacion_prom_TC_X',
'llave_lag',
'max_cupo_sector_hip_sin_popu_smlv',
'max_cupo_sector_hip_sin_popudif',
'max_cupo_tdc_sin_popular_smlv',
'max_cupo_tdc_sin_populardif',
'max_dias_procesamiento',
'maxcuposectorbancasin_popular_smlv',
'maxcuposectorbancasin_populardif',
'meses_ultim_aper_CE',
'meses_ultim_aper_CH',
'meses_ultim_aper_LB',
'meses_ultim_aper_LE',
'meses_ultim_aper_NV',
'meses_ultim_aper_TC',
'meses_ultim_aper_TC_T',
'meses_ultim_aper_TC_X',
'meses_ultim_aper_tot',
'mora_cierre_ult_mes_expr',
'mora_cierre_ult_mes_total',
'mora_max_tot',
'mora_max_tot_ult3_meses',
'mora_max_tot_ult6_meses',
'mora_max_ult3_meses_expr',
'mora_max_ult3_meses_total',
'mora_max_ult6_meses_expr',
'mora_max_ult6_meses_total',
'mora_maxLB',
'moramax_mercado',
'moramax_mercado_2',
'moramax_mercado_2_was_is_mantiene',
'moramax_mercado_lag1',
'n_novaciones',
'num_act_utl_meses',
'num_aper_ultim_12meses_CE',
'num_aper_ultim_12meses_LB',
'num_aper_ultim_12meses_LE',
'num_aper_ultim_12meses_TC',
'num_aper_ultim_12meses_TC_T',
'num_aper_ultim_12meses_TC_X',
'num_aper_ultim_12meses_tot',
'num_aper_ultim_1meses_LB',
'num_aper_ultim_1meses_tot',
'num_aper_ultim_3meses_CE',
'num_aper_ultim_3meses_LB',
'num_aper_ultim_3meses_TC',
'num_aper_ultim_3meses_TC_X',
'num_aper_ultim_3meses_tot',
'num_aper_ultim_6meses_CE',
'num_aper_ultim_6meses_LB',
'num_aper_ultim_6meses_TC',
'num_aper_ultim_6meses_TC_X',
'num_aper_ultim_6meses_tot',
'num_camp_ult_meses',
'num_lib_solicitadas',
'num_meses_ult_actv',
'num_meses_ult_camp',
'num_no_aceptado',
'num_obl_aldia_CE',
'num_obl_aldia_CH',
'num_obl_aldia_LB',
'num_obl_aldia_LE',
'num_obl_aldia_NV',
'num_obl_aldia_SO',
'num_obl_aldia_TC',
'num_obl_aldia_TC_T',
'num_obl_aldia_TC_X',
'num_obl_aldia_tot',
'num_obl_mora1_30_LB',
'num_obl_mora1_30_tot',
'num_obl_vig_CE',
'num_obl_vig_CH',
'num_obl_vig_LB',
'num_obl_vig_LE',
'num_obl_vig_NV',
'num_obl_vig_SO',
'num_obl_vig_TC',
'num_obl_vig_TC_T',
'num_obl_vig_TC_X',
'num_obl_vig_tot',
'num_tdc_vigentes_sin_popular',
'num_tdc_vigentes_sin_populardif',
'num_tx_mes_ant',
'num_tx_mes_ant_expr',
'num_tx_mes_ant_trad',
'num_tx_ult_1mes',
'num_tx_ult_1mes_ahr',
'num_tx_ult_1mes_cdt',
'num_tx_ult_1mes_cte',
'num_tx_ult_1mes_expr',
'num_tx_ult_1mes_pasv',
'num_tx_ult_1mes_trad',
'num_tx_ult_2meses',
'num_tx_ult_2meses_ahr',
'num_tx_ult_2meses_cdt',
'num_tx_ult_2meses_cte',
'num_tx_ult_2meses_expr',
'num_tx_ult_2meses_pasv',
'num_tx_ult_2meses_trad',
'num_tx_ult_3meses',
'num_tx_ult_3meses_ahr',
'num_tx_ult_3meses_cdt',
'num_tx_ult_3meses_cte',
'num_tx_ult_3meses_expr',
'num_tx_ult_3meses_pasv',
'num_tx_ult_3meses_trad',
'num_tx_ult_4meses',
'num_tx_ult_4meses_ahr',
'num_tx_ult_4meses_cdt',
'num_tx_ult_4meses_cte',
'num_tx_ult_4meses_expr',
'num_tx_ult_4meses_pasv',
'num_tx_ult_4meses_trad',
'num_tx_ult_6meses',
'num_tx_ult_6meses_ahr',
'num_tx_ult_6meses_cdt',
'num_tx_ult_6meses_cte',
'num_tx_ult_6meses_expr',
'num_tx_ult_6meses_pasv',
'num_tx_ult_6meses_trad',
'numero_celulares_telcos',
'numero_celulares_telcosdif',
'numero_creditos_cb',
'numero_creditos_cbdif',
'numero_creditos_cf',
'numero_creditos_cfdif',
'numero_creditos_codeudores',
'numero_creditos_codeudoresdif',
'numero_creditos_cooperativas',
'numero_creditos_cooperativasdif',
'numero_creditos_cv',
'numero_creditos_cvdif',
'numero_creditos_sr',
'numero_creditos_srdif',
'numero_obligaciones_activas',
'numero_obligaciones_activasdif',
'numero_tdc',
'numero_tdcdif',
'numoblvigensector_hip_sin_popu',
'numoblvigensector_hip_sin_popudif',
'numoblvigensectorbancasin_popu',
'numoblvigensectorbancasin_popudif',
'obliga_al_dia_cartera_actual',
'obliga_al_dia_cartera_actualdif',
'obliga_mora_120_cartera_actual',
'obliga_mora_120_cartera_actualdif',
'obliga_mora_30_cartera_actual',
'obliga_mora_30_cartera_actualdif',
'obliga_mora_60_cartera_actual',
'obliga_mora_60_cartera_actualdif',
'obliga_mora_90_cartera_actual',
'obliga_mora_90_cartera_actualdif',
'pasv_antig_total',
'pasv_dias_desde_ultima_trans',
'pasv_num_ctas_vig_ca',
'pasv_num_ctas_vig_cc',
'pasv_num_ctas_vig_cdt',
'pasv_num_ctas_vig_tot',
'pasv_num_meses_ult_apertura',
'pasv_saldo_ca_1mes_atras',
'pasv_saldo_ca_3mes_atras',
'pasv_saldo_ca_6mes_atras',
'pasv_saldo_ca_fin_mes_smlv',
'pasv_saldo_cc_1mes_atras',
'pasv_saldo_cc_3mes_atras',
'pasv_saldo_cc_6mes_atras',
'pasv_saldo_cc_fin_mes_smlv',
'pasv_saldo_cdt_1mes_atras',
'pasv_saldo_cdt_3mes_atras',
'pasv_saldo_cdt_6mes_atras',
'pasv_saldo_cdt_fin_mes_smlv',
'pasv_saldo_max_ca_fin_mes_smlv',
'pasv_saldo_max_cc_fin_mes_smlv',
'pasv_saldo_max_cdt_fin_mes_smlv',
'pasv_saldo_max_fin_mes_smlv',
'pasv_saldo_min_ca_fin_mes_smlv',
'pasv_saldo_min_cc_fin_mes_smlv',
'pasv_saldo_min_cdt_fin_mes_smlv',
'pasv_saldo_min_fin_mes_smlv',
'pasv_saldo_tot_fin_mes_smlv',
'pasv_saldo_total_1mes_atras',
'pasv_saldo_total_3mes_atras',
'pasv_saldo_total_6mes_atras',
'porc_obl_aldia_LB',
'porc_obl_aldia_tot',
'porc_obl_mora1_30_LB',
'porc_obl_mora1_30_tot',
'porcentaje_utilizacion',
'porcentaje_utilizaciondif',
'prom_cuot_ult_3meses',
'prom_cuot_ult_3meses_trad',
'prom_cuot_ult_4meses',
'prom_cuot_ult_4meses_trad',
'prom_cuot_ult_6meses',
'prom_cuot_ult_6meses_trad',
'prom_cupo_aprob',
'prom_cupo_sector_hip_sin_popu_smlv',
'prom_cupo_sector_hip_sin_popudif',
'prom_dias_procesamiento',
'prom_monto_aprobado',
'prom_monto_desembolsado',
'prom_monto_novado',
'prom_monto_solicitado',
'prom_mov_mes_ant',
'prom_mov_mes_ant_expr',
'prom_mov_mes_ant_la14',
'prom_mov_mes_ant_trad',
'prom_mov_ult_1mes',
'prom_mov_ult_1mes_expr',
'prom_mov_ult_1mes_trad',
'prom_mov_ult_2meses',
'prom_mov_ult_2meses_expr',
'prom_mov_ult_2meses_trad',
'prom_mov_ult_3meses',
'prom_mov_ult_3meses_expr',
'prom_mov_ult_3meses_trad',
'prom_mov_ult_4meses',
'prom_mov_ult_4meses_expr',
'prom_mov_ult_4meses_trad',
'prom_mov_ult_6meses',
'prom_mov_ult_6meses_expr',
'prom_mov_ult_6meses_trad',
'prom_n_cuotas',
'prom_prop_desembolso',
'prom_tasa',
'prom_tx_ult_1mes',
'prom_tx_ult_1mes_expr',
'prom_tx_ult_1mes_trad',
'prom_tx_ult_1meses_ahr',
'prom_tx_ult_1meses_cdt',
'prom_tx_ult_1meses_cte',
'prom_tx_ult_1meses_pasv',
'prom_tx_ult_2meses',
'prom_tx_ult_2meses_ahr',
'prom_tx_ult_2meses_cdt',
'prom_tx_ult_2meses_cte',
'prom_tx_ult_2meses_expr',
'prom_tx_ult_2meses_pasv',
'prom_tx_ult_2meses_trad',
'prom_tx_ult_3meses',
'prom_tx_ult_3meses_ahr',
'prom_tx_ult_3meses_cdt',
'prom_tx_ult_3meses_cte',
'prom_tx_ult_3meses_expr',
'prom_tx_ult_3meses_pasv',
'prom_tx_ult_3meses_trad',
'prom_tx_ult_4meses',
'prom_tx_ult_4meses_ahr',
'prom_tx_ult_4meses_cdt',
'prom_tx_ult_4meses_cte',
'prom_tx_ult_4meses_expr',
'prom_tx_ult_4meses_pasv',
'prom_tx_ult_4meses_trad',
'prom_tx_ult_6meses',
'prom_tx_ult_6meses_ahr',
'prom_tx_ult_6meses_cdt',
'prom_tx_ult_6meses_cte',
'prom_tx_ult_6meses_expr',
'prom_tx_ult_6meses_pasv',
'prom_tx_ult_6meses_trad',
'prom_valor_cuota',
'promcuposectorbancasin_popular_smlv',
'promcuposectorbancasin_populardif',
'promedio_cupo_tdc_sin_popular_smlv',
'promedio_cupo_tdc_sin_populardif',
'quanto_mod',
'rango_0',
'rango_1',
'rango_2',
'rango_3',
'rango_4',
'rango_5',
'rango_6',
'saldo_max_CE_SMLV',
'saldo_max_CH_SMLV',
'saldo_max_LB_SMLV',
'saldo_max_LE_SMLV',
'saldo_max_NV_SMLV',
'saldo_max_TC_SMLV',
'saldo_max_TC_T_SMLV',
'saldo_max_TC_X_SMLV',
'saldo_max_tot_SMLV',
'saldo_mercado_smlv',
'saldo_mercadodif',
'saldo_prom_CE_SMLV',
'saldo_prom_CH_SMLV',
'saldo_prom_LB_SMLV',
'saldo_prom_LE_SMLV',
'saldo_prom_NV_SMLV',
'saldo_prom_TC_SMLV',
'saldo_prom_TC_T_SMLV',
'saldo_prom_TC_X_SMLV',
'saldo_prom_tot_SMLV',
'saldo_prom_ult3_meses_expr',
'saldo_prom_ult3_meses_total',
'saldo_prom_ult3_meses_trad',
'saldo_prom_ult6_meses_expr',
'saldo_prom_ult6_meses_total',
'saldo_prom_ult6_meses_trad',
'saldo_prom_ult7_meses_expr',
'saldo_prom_ult7_meses_total',
'saldo_prom_ult7_meses_trad',
'saldo_tot_CE_SMLV',
'saldo_tot_CE_SMLV_ult3_meses',
'saldo_tot_CE_SMLV_ult7_meses',
'saldo_tot_CH_SMLV',
'saldo_tot_CH_SMLV_ult3_meses',
'saldo_tot_CH_SMLV_ult7_meses',
'saldo_tot_LB_SMLV',
'saldo_tot_LB_SMLV_ult3_meses',
'saldo_tot_LB_SMLV_ult7_meses',
'saldo_tot_LE_SMLV',
'saldo_tot_LE_SMLV_ult3_meses',
'saldo_tot_LE_SMLV_ult7_meses',
'saldo_tot_NV_SMLV',
'saldo_tot_NV_SMLV_ult3_meses',
'saldo_tot_NV_SMLV_ult7_meses',
'saldo_tot_SO_SMLV_ult7_meses',
'saldo_tot_TC_SMLV',
'saldo_tot_TC_SMLV_ult3_meses',
'saldo_tot_TC_SMLV_ult7_meses',
'saldo_tot_TC_T_SMLV',
'saldo_tot_TC_T_SMLV_ult3_meses',
'saldo_tot_TC_T_SMLV_ult7_meses',
'saldo_tot_TC_X_SMLV',
'saldo_tot_TC_X_SMLV_ult3_meses',
'saldo_tot_TC_X_SMLV_ult7_meses',
'saldo_tot_tot_SMLV',
'saldo_tot_tot_SMLV_ult3_meses',
'saldo_tot_tot_SMLV_ult7_meses',
'sum_tx_linea_ahr',
'sum_tx_linea_cdt',
'sum_tx_linea_cte',
'sum_tx_linea_pasv',
'sum_tx_nolinea_ahr',
'sum_tx_nolinea_cdt',
'sum_tx_nolinea_cte',
'sum_tx_nolinea_pasv',
'tdc_altura_maxima_de_mora',
'tdc_altura_maxima_de_mora_lag1',
'tiene_carbancos',
'tiene_cartfinanciera',
'tiene_codeudor',
'tiene_hipotecario',
'tiene_sectorcoop',
'tiene_sectorreal',
'timestamp',
'total_consultas_ult_6_meses',
'total_consultas_ult_6_mesesdif',
'ult_año_moras_120_cartera_hist',
'ult_año_moras_120_cartera_histdif',
'ult_año_moras_30_cartera_hist',
'ult_año_moras_30_cartera_histdif',
'ult_año_moras_60_cartera_hist',
'ult_año_moras_60_cartera_histdif',
'ult_año_moras_90_cartera_hist',
'ult_año_moras_90_cartera_histdif',
'util_prom_ult3_meses_expr',
'util_prom_ult3_meses_total',
'util_prom_ult3_meses_trad',
'util_prom_ult6_meses_expr',
'util_prom_ult6_meses_total',
'util_prom_ult6_meses_trad',
'util_prom_ult9_meses_expr',
'util_prom_ult9_meses_total',
'util_prom_ult9_meses_trad',
'util_sector_banca_sin_popular',
'util_sector_banca_sin_populardif',
'util_sector_hip_sin_popular',
'util_sector_hip_sin_populardif',
'utilizacion_max_TC',
'utilizacion_max_TC_T',
'utilizacion_max_TC_X',
'utilizacion_min_TC',
'utilizacion_min_TC_T',
'utilizacion_min_TC_X',
'utilizacion_prom_TC',
'utilizacion_prom_TC_T',
'utilizacion_prom_TC_T_ult3_meses',
'utilizacion_prom_TC_T_ult9_meses',
'utilizacion_prom_TC_ult3_meses',
'utilizacion_prom_TC_ult9_meses',
'utilizacion_prom_TC_X',
'utilizacion_prom_TC_X_ult3_meses',
'utilizacion_prom_TC_X_ult9_meses',
'utilizacion_sin_popular',
'utilizacion_sin_populardif',
'val_cuo_sector_banca_sin_popu',
'val_cuo_sector_banca_sin_popudif',
'val_cuotas_sector_hip_sin_popu_smlv',
'val_cuotas_sector_hip_sin_popudif',
'val_mora_sector_hip_sin_popu',
'val_mora_sector_hip_sin_popudif',
'val_morasectorbancasin_popular',
'val_morasectorbancasin_populardif',
'val_util_sector_hip_sin_pop',
'val_util_sector_hip_sin_popdif',
'valor_cuotas_cb_smlv',
'valor_cuotas_cbdif',
'valor_cuotas_celulares_telcos_smlv',
'valor_cuotas_celulares_telcosdif',
'valor_cuotas_cf_smlv',
'valor_cuotas_cfdif',
'valor_cuotas_codeudores_smlv',
'valor_cuotas_codeudoresdif',
'valor_cuotas_cooperativas_smlv',
'valor_cuotas_cooperativasdif',
'valor_cuotas_cv_smlv',
'valor_cuotas_cvdif',
'valor_cuotas_sin_popular_smlv',
'valor_cuotas_sin_populardif',
'valor_cuotas_smlv',
'valor_cuotas_sr_smlv',
'valor_cuotas_srdif',
'valor_cuotasdif',
'valor_cupos_smlv',
'valor_cuposdif',
'valor_en_mora_sin_popular_smlv',
'valor_en_mora_sin_populardif',
'valor_inicial_cb_smlv',
'valor_inicial_cbdif',
'valor_inicial_cf_smlv',
'valor_inicial_cfdif',
'valor_inicial_cooperativas_smlv',
'valor_inicial_cooperativasdif',
'valor_inicial_cv_smlv',
'valor_inicial_cvdif',
'valor_inicial_sr_smlv',
'valor_inicial_srdif',
'valor_mora_cb_smlv',
'valor_mora_cbdif',
'valor_mora_cf_smlv',
'valor_mora_cfdif',
'valor_mora_codeudores_smlv',
'valor_mora_codeudoresdif',
'valor_mora_cooperativas_smlv',
'valor_mora_cooperativasdif',
'valor_mora_cv_smlv',
'valor_mora_cvdif',
'valor_mora_smlv',
'valor_mora_sr_smlv',
'valor_mora_srdif',
'valor_mora_telcos_smlv',
'valor_mora_telcosdif',
'valor_moradif',
'valor_mov_ult_1mes',
'valor_mov_ult_1mes_expr',
'valor_mov_ult_1mes_trad',
'valor_mov_ult_2meses',
'valor_mov_ult_2meses_expr',
'valor_mov_ult_2meses_trad',
'valor_mov_ult_3meses',
'valor_mov_ult_3meses_expr',
'valor_mov_ult_3meses_trad',
'valor_mov_ult_4meses',
'valor_mov_ult_4meses_expr',
'valor_mov_ult_4meses_trad',
'valor_mov_ult_6meses',
'valor_mov_ult_6meses_expr',
'valor_mov_ult_6meses_trad',
'valor_saldo_cb_smlv',
'valor_saldo_cbdif',
'valor_saldo_cf_smlv',
'valor_saldo_cfdif',
'valor_saldo_codeudores_smlv',
'valor_saldo_codeudoresdif',
'valor_saldo_cooperativas_smlv',
'valor_saldo_cooperativasdif',
'valor_saldo_cv_smlv',
'valor_saldo_cvdif',
'valor_saldo_sr_smlv',
'valor_saldo_srdif',
'valor_utilisectorbancasin_popu_smlv',
'valor_utilisectorbancasin_popudif',
'valor_utilizado_sin_popular_smlv',
'valor_utilizado_sin_populardif',
'valor_utilizado_smlv',
'valor_utilizadodif',
'var_final'
]

In [57]:
variables_continuas_perf=['acep_oferta_prev',
'num_act_utl_meses',
'num_camp_ult_meses',
'num_meses_ult_actv',
'num_meses_ult_camp',
'num_no_aceptado',
'prom_cupo_aprob',
'acierta_a_financiero',
'cartera_banca_alt_max_de_mora',
'cartera_castig_cartera_actual',
'cartera_recupe_cartera_hist',
'ctas_ctes_act_cta_banca',
'ctas_de_ahorro_act_ctas_banca',
'ctas_embargadas_ctas_bancarias',
'ctas_saldadas_ctas_banca',
'endeudamiento',
'estado_consulta',
'fecantiapersectorbancasinpopu',
'fecha_mas_antigua_apertura',
'fec_masantapersectorhipsin_pop',
'fec_mas_anti_aper_tdc_sin_popu',
'numero_celulares_telcos',
'numero_creditos_cb',
'numero_creditos_cf',
'numero_creditos_codeudores',
'numero_creditos_cooperativas',
'numero_creditos_cv',
'numero_creditos_sr',
'numero_obligaciones_activas',
'numero_tdc',
'numoblvigensectorbancasin_popu',
'numoblvigensector_hip_sin_popu',
'num_tdc_vigentes_sin_popular',
'obliga_al_dia_cartera_actual',
'obliga_mora_120_cartera_actual',
'obliga_mora_30_cartera_actual',
'porcentaje_utilizacion',
'quanto_mod',
'rango_1',
'rango_2',
'rango_3',
'rango_4',
'rango_5',
'rango_6',
'tdc_altura_maxima_de_mora',
'total_consultas_ult_6_meses',
'ult_año_moras_120_cartera_hist',
'ult_año_moras_30_cartera_hist',
'ult_año_moras_60_cartera_hist',
'utilizacion_sin_popular',
'util_sector_banca_sin_popular',
'util_sector_hip_sin_popular',
'val_cuo_sector_banca_sin_popu',
'val_morasectorbancasin_popular',
'val_util_sector_hip_sin_pop',
'moramax_mercado',
'tiene_carbancos',
'tiene_hipotecario',
'tiene_cartfinanciera',
'tiene_sectorreal',
'tiene_sectorcoop',
'tiene_codeudor',
'porcentaje_utilizaciondif',
'numero_obligaciones_activasdif',
'numero_creditos_cbdif',
'valor_inicial_cbdif',
'valor_saldo_cbdif',
'valor_cuotas_cbdif',
'valor_mora_cbdif',
'numero_creditos_cvdif',
'valor_inicial_cvdif',
'valor_saldo_cvdif',
'valor_cuotas_cvdif',
'numero_creditos_cfdif',
'valor_inicial_cfdif',
'valor_saldo_cfdif',
'valor_cuotas_cfdif',
'valor_mora_cfdif',
'numero_tdcdif',
'valor_cuposdif',
'valor_utilizadodif',
'valor_cuotasdif',
'valor_moradif',
'numero_creditos_srdif',
'valor_inicial_srdif',
'valor_saldo_srdif',
'valor_cuotas_srdif',
'numero_celulares_telcosdif',
'valor_cuotas_celulares_telcosdif',
'numero_creditos_cooperativasdif',
'valor_inicial_cooperativasdif',
'valor_saldo_cooperativasdif',
'valor_cuotas_cooperativasdif',
'numero_creditos_codeudoresdif',
'valor_saldo_codeudoresdif',
'valor_cuotas_codeudoresdif',
'valor_mora_codeudoresdif',
'obliga_al_dia_cartera_actualdif',
'obliga_mora_30_cartera_actualdif',
'obliga_mora_120_cartera_actualdif',
'cartera_castig_cartera_actualdif',
'ult_año_moras_30_cartera_histdif',
'ult_año_moras_60_cartera_histdif',
'ult_año_moras_120_cartera_histdif',
'cartera_recupe_cartera_histdif',
'ctas_de_ahorro_act_ctas_bancadif',
'ctas_ctes_act_cta_bancadif',
'ctas_saldadas_ctas_bancadif',
'total_consultas_ult_6_mesesdif',
'num_tdc_vigentes_sin_populardif',
'cupo_sin_populardif',
'max_cupo_tdc_sin_populardif',
'promedio_cupo_tdc_sin_populardif',
'valor_utilizado_sin_populardif',
'utilizacion_sin_populardif',
'valor_cuotas_sin_populardif',
'valor_en_mora_sin_populardif',
'numoblvigensectorbancasin_popudif',
'cuposectorbancario_sin_populardif',
'maxcuposectorbancasin_populardif',
'promcuposectorbancasin_populardif',
'valor_utilisectorbancasin_popudif',
'util_sector_banca_sin_populardif',
'val_cuo_sector_banca_sin_popudif',
'val_morasectorbancasin_populardif',
'numoblvigensector_hip_sin_popudif',
'cupo_sector_hip_sin_populardif',
'max_cupo_sector_hip_sin_popudif',
'prom_cupo_sector_hip_sin_popudif',
'val_util_sector_hip_sin_popdif',
'util_sector_hip_sin_populardif',
'val_cuotas_sector_hip_sin_popudif',
'saldo_mercadodif',
'cuota_mercadodif',
'cupo_mercadodif',
'valor_saldo_cb_smlv',
'valor_saldo_cv_smlv',
'valor_saldo_cf_smlv',
'valor_saldo_sr_smlv',
'valor_saldo_cooperativas_smlv',
'valor_saldo_codeudores_smlv',
'saldo_mercado_smlv',
'valor_cupos_smlv',
'cupo_sin_popular_smlv',
'max_cupo_tdc_sin_popular_smlv',
'promedio_cupo_tdc_sin_popular_smlv',
'cuposectorbancario_sin_popular_smlv',
'maxcuposectorbancasin_popular_smlv',
'promcuposectorbancasin_popular_smlv',
'cupo_sector_hip_sin_popular_smlv',
'max_cupo_sector_hip_sin_popu_smlv',
'prom_cupo_sector_hip_sin_popu_smlv',
'cupo_mercado_smlv',
'valor_inicial_cb_smlv',
'valor_inicial_cv_smlv',
'valor_inicial_cf_smlv',
'valor_inicial_sr_smlv',
'valor_inicial_cooperativas_smlv',
'valor_cuotas_cb_smlv',
'valor_cuotas_cv_smlv',
'valor_cuotas_cf_smlv',
'valor_cuotas_smlv',
'valor_cuotas_sr_smlv',
'valor_cuotas_celulares_telcos_smlv',
'valor_cuotas_cooperativas_smlv',
'valor_cuotas_codeudores_smlv',
'valor_cuotas_sin_popular_smlv',
'val_cuotas_sector_hip_sin_popu_smlv',
'cuota_mercado_smlv',
'valor_mora_cb_smlv',
'valor_mora_cf_smlv',
'valor_utilizado_smlv',
'valor_mora_smlv',
'valor_mora_codeudores_smlv',
'valor_utilizado_sin_popular_smlv',
'valor_en_mora_sin_popular_smlv',
'valor_utilisectorbancasin_popu_smlv',
'moramax_mercado_2',
'tdc_altura_maxima_de_mora_lag1',
'moramax_mercado_lag1',
'moramax_mercado_2_was_is_mantiene',
'num_aper_ultim_1meses_LB',
'num_aper_ultim_1meses_TC',
'num_aper_ultim_1meses_NV',
'num_aper_ultim_3meses_LB',
'num_aper_ultim_3meses_TC',
'num_aper_ultim_3meses_NV',
'num_aper_ultim_3meses_CE',
'num_aper_ultim_6meses_LB',
'num_aper_ultim_6meses_TC',
'num_aper_ultim_6meses_NV',
'num_aper_ultim_6meses_CE',
'num_aper_ultim_12meses_LB',
'num_aper_ultim_12meses_TC',
'num_aper_ultim_12meses_NV',
'num_aper_ultim_12meses_CE',
'num_aper_ultim_12meses_LE',
'num_aper_ultim_1meses_TC_X',
'num_aper_ultim_1meses_TC_T',
'num_aper_ultim_3meses_TC_X',
'num_aper_ultim_3meses_TC_T',
'num_aper_ultim_6meses_TC_X',
'num_aper_ultim_6meses_TC_T',
'num_aper_ultim_12meses_TC_X',
'num_aper_ultim_12meses_TC_T',
'num_aper_ultim_1meses_tot',
'num_aper_ultim_3meses_tot',
'num_aper_ultim_6meses_tot',
'num_aper_ultim_12meses_tot',
'meses_ultim_aper_LB',
'meses_ultim_aper_TC',
'meses_ultim_aper_NV',
'meses_ultim_aper_CE',
'meses_ultim_aper_LE',
'meses_ultim_aper_SO',
'meses_ultim_aper_TC_X',
'meses_ultim_aper_TC_T',
'meses_ultim_aper_tot',
'antiguedad_tot_conlibranza',
'antiguedad_tot_sinlibranza',
'antiguedad_TDC_T',
'antiguedad_TDC_X',
'num_obl_vig_LB',
'num_obl_vig_TC',
'num_obl_vig_NV',
'num_obl_vig_CE',
'num_obl_vig_LE',
'num_obl_vig_SO',
'num_obl_vig_TC_X',
'num_obl_vig_TC_T',
'num_obl_vig_tot',
'num_obl_aldia_LB',
'porc_obl_aldia_LB',
'num_obl_aldia_TC',
'porc_obl_aldia_TC',
'num_obl_mora1_30_TC',
'porc_obl_mora1_30_TC',
'num_obl_aldia_NV',
'porc_obl_aldia_NV',
'num_obl_mora1_30_NV',
'porc_obl_mora1_30_NV',
'num_obl_mora31_60_NV',
'porc_obl_mora31_60_NV',
'num_obl_aldia_CE',
'porc_obl_aldia_CE',
'num_obl_aldia_LE',
'porc_obl_aldia_LE',
'num_obl_mora1_30_LE',
'porc_obl_mora1_30_LE',
'num_obl_aldia_SO',
'porc_obl_aldia_SO',
'num_obl_aldia_TC_X',
'porc_obl_aldia_TC_X',
'num_obl_mora1_30_TC_X',
'porc_obl_mora1_30_TC_X',
'num_obl_aldia_TC_T',
'porc_obl_aldia_TC_T',
'num_obl_aldia_tot',
'porc_obl_aldia_tot',
'num_obl_mora1_30_tot',
'porc_obl_mora1_30_tot',
'num_obl_mora31_60_tot',
'porc_obl_mora31_60_tot',
'utilizacion_prom_TC_X',
'utilizacion_min_TC_X',
'utilizacion_max_TC_X',
'utilizacion_prom_TC_T',
'utilizacion_min_TC_T',
'utilizacion_max_TC_T',
'utilizacion_prom_TC',
'utilizacion_min_TC',
'utilizacion_max_TC',
'amortizacion_prom_LB',
'amortizacion_min_LB',
'amortizacion_max_LB',
'amortizacion_prom_NV',
'amortizacion_min_NV',
'amortizacion_max_NV',
'amortizacion_prom_CE',
'amortizacion_min_CE',
'amortizacion_max_CE',
'amortizacion_prom_LE',
'amortizacion_min_LE',
'amortizacion_max_LE',
'amortizacion_prom_tot',
'amortizacion_min_tot',
'amortizacion_max_tot',
'mora_maxTC',
'mora_maxNV',
'mora_maxLE',
'mora_maxTC_X',
'mora_max_tot',
'saldo_tot_LB_SMLV',
'saldo_max_LB_SMLV',
'saldo_prom_LB_SMLV',
'saldo_tot_TC_SMLV',
'saldo_max_TC_SMLV',
'saldo_prom_TC_SMLV',
'saldo_tot_NV_SMLV',
'saldo_max_NV_SMLV',
'saldo_prom_NV_SMLV',
'saldo_tot_CE_SMLV',
'saldo_max_CE_SMLV',
'saldo_prom_CE_SMLV',
'saldo_tot_LE_SMLV',
'saldo_max_LE_SMLV',
'saldo_prom_LE_SMLV',
'saldo_tot_SO_SMLV',
'saldo_max_SO_SMLV',
'saldo_prom_SO_SMLV',
'saldo_tot_TC_X_SMLV',
'saldo_max_TC_X_SMLV',
'saldo_prom_TC_X_SMLV',
'saldo_tot_TC_T_SMLV',
'saldo_max_TC_T_SMLV',
'saldo_prom_TC_T_SMLV',
'saldo_tot_tot_SMLV',
'saldo_max_tot_SMLV',
'saldo_prom_tot_SMLV',
'cupo_tot_LB_SMLV',
'cupo_max_LB_SMLV',
'cupo_prom_LB_SMLV',
'cupo_tot_TC_SMLV',
'cupo_max_TC_SMLV',
'cupo_prom_TC_SMLV',
'cupo_tot_NV_SMLV',
'cupo_max_NV_SMLV',
'cupo_prom_NV_SMLV',
'cupo_tot_CE_SMLV',
'cupo_max_CE_SMLV',
'cupo_prom_CE_SMLV',
'cupo_tot_LE_SMLV',
'cupo_max_LE_SMLV',
'cupo_prom_LE_SMLV',
'cupo_tot_SO_SMLV',
'cupo_max_SO_SMLV',
'cupo_prom_SO_SMLV',
'cupo_tot_TC_X_SMLV',
'cupo_max_TC_X_SMLV',
'cupo_prom_TC_X_SMLV',
'cupo_tot_TC_T_SMLV',
'cupo_max_TC_T_SMLV',
'cupo_prom_TC_T_SMLV',
'cupo_tot_tot_SMLV',
'cupo_max_tot_SMLV',
'cupo_prom_tot_SMLV',
'cuota_tot_LB_SMLV',
'cuota_max_LB_SMLV',
'cuota_prom_LB_SMLV',
'cuota_tot_TC_SMLV',
'cuota_max_TC_SMLV',
'cuota_prom_TC_SMLV',
'cuota_tot_NV_SMLV',
'cuota_max_NV_SMLV',
'cuota_prom_NV_SMLV',
'cuota_tot_CE_SMLV',
'cuota_max_CE_SMLV',
'cuota_prom_CE_SMLV',
'cuota_tot_LE_SMLV',
'cuota_max_LE_SMLV',
'cuota_prom_LE_SMLV',
'cuota_tot_TC_X_SMLV',
'cuota_max_TC_X_SMLV',
'cuota_prom_TC_X_SMLV',
'cuota_tot_TC_T_SMLV',
'cuota_max_TC_T_SMLV',
'cuota_prom_TC_T_SMLV',
'cuota_tot_tot_SMLV',
'cuota_max_tot_SMLV',
'cuota_prom_tot_SMLV',
'mora_max_tot_ult3_meses',
'mora_max_tot_ult6_meses',
'saldo_tot_LB_SMLV_ult3_meses',
'saldo_tot_LB_SMLV_ult7_meses',
'saldo_tot_TC_SMLV_ult3_meses',
'saldo_tot_TC_SMLV_ult7_meses',
'saldo_tot_NV_SMLV_ult3_meses',
'saldo_tot_NV_SMLV_ult7_meses',
'saldo_tot_CH_SMLV_ult3_meses',
'saldo_tot_CH_SMLV_ult7_meses',
'saldo_tot_CE_SMLV_ult3_meses',
'saldo_tot_CE_SMLV_ult7_meses',
'saldo_tot_LE_SMLV_ult3_meses',
'saldo_tot_LE_SMLV_ult7_meses',
'saldo_tot_SO_SMLV_ult3_meses',
'saldo_tot_SO_SMLV_ult7_meses',
'saldo_tot_TC_X_SMLV_ult3_meses',
'saldo_tot_TC_X_SMLV_ult7_meses',
'saldo_tot_TC_T_SMLV_ult3_meses',
'saldo_tot_TC_T_SMLV_ult7_meses',
'saldo_tot_tot_SMLV_ult3_meses',
'saldo_tot_tot_SMLV_ult7_meses',
'cuota_tot_LB_SMLV_ult3_meses',
'cuota_tot_LB_SMLV_ult8_meses',
'cuota_tot_TC_SMLV_ult3_meses',
'cuota_tot_TC_SMLV_ult8_meses',
'cuota_tot_NV_SMLV_ult3_meses',
'cuota_tot_NV_SMLV_ult8_meses',
'cuota_tot_CH_SMLV_ult3_meses',
'cuota_tot_CH_SMLV_ult8_meses',
'cuota_tot_CE_SMLV_ult3_meses',
'cuota_tot_CE_SMLV_ult8_meses',
'cuota_tot_LE_SMLV_ult3_meses',
'cuota_tot_LE_SMLV_ult8_meses',
'cuota_tot_TC_X_SMLV_ult3_meses',
'cuota_tot_TC_X_SMLV_ult8_meses',
'cuota_tot_TC_T_SMLV_ult3_meses',
'cuota_tot_TC_T_SMLV_ult8_meses',
'cuota_tot_tot_SMLV_ult3_meses',
'cuota_tot_tot_SMLV_ult8_meses',
'cupo_tot_LB_SMLV_ult3_meses',
'cupo_tot_LB_SMLV_ult9_meses',
'cupo_tot_TC_SMLV_ult3_meses',
'cupo_tot_TC_SMLV_ult9_meses',
'cupo_tot_NV_SMLV_ult3_meses',
'cupo_tot_NV_SMLV_ult9_meses',
'cupo_tot_CH_SMLV_ult3_meses',
'cupo_tot_CH_SMLV_ult9_meses',
'cupo_tot_CE_SMLV_ult3_meses',
'cupo_tot_CE_SMLV_ult9_meses',
'cupo_tot_LE_SMLV_ult3_meses',
'cupo_tot_LE_SMLV_ult9_meses',
'cupo_tot_SO_SMLV_ult3_meses',
'cupo_tot_SO_SMLV_ult9_meses',
'cupo_tot_TC_X_SMLV_ult3_meses',
'cupo_tot_TC_X_SMLV_ult9_meses',
'cupo_tot_TC_T_SMLV_ult3_meses',
'cupo_tot_TC_T_SMLV_ult9_meses',
'cupo_tot_tot_SMLV_ult3_meses',
'cupo_tot_tot_SMLV_ult9_meses',
'utilizacion_prom_TC_X_ult3_meses',
'utilizacion_prom_TC_X_ult9_meses',
'utilizacion_prom_TC_T_ult3_meses',
'utilizacion_prom_TC_T_ult9_meses',
'utilizacion_prom_TC_ult3_meses',
'utilizacion_prom_TC_ult9_meses',
'amortizacion_prom_LB_ult3_meses',
'amortizacion_prom_LB_ult9_meses',
'amortizacion_prom_NV_ult3_meses',
'amortizacion_prom_NV_ult9_meses',
'amortizacion_prom_CH_ult3_meses',
'amortizacion_prom_CH_ult9_meses',
'amortizacion_prom_CE_ult3_meses',
'amortizacion_prom_CE_ult9_meses',
'amortizacion_prom_LE_ult3_meses',
'amortizacion_prom_LE_ult9_meses',
'amortizacion_prom_tot_ult3_meses',
'amortizacion_prom_tot_ult9_meses',
'dif_porc_t_1_mora_maxTC',
'dif_porc_t_6_mora_maxTC',
'dif_porc_t_1_mora_maxNV',
'dif_porc_t_6_mora_maxNV',
'dif_porc_t_1_mora_maxLE',
'dif_porc_t_6_mora_maxLE',
'dif_porc_t_1_mora_maxTC_X',
'dif_porc_t_6_mora_maxTC_X',
'dif_porc_t_1_mora_max_tot',
'dif_porc_t_6_mora_max_tot',
'dif_porc_t_1_saldo_tot_LB_SMLV',
'dif_porc_t_6_saldo_tot_LB_SMLV',
'dif_porc_t_1_saldo_tot_TC_SMLV',
'dif_porc_t_6_saldo_tot_TC_SMLV',
'dif_porc_t_1_saldo_tot_NV_SMLV',
'dif_porc_t_6_saldo_tot_NV_SMLV',
'dif_porc_t_1_saldo_tot_SO_SMLV',
'dif_porc_t_6_saldo_tot_SO_SMLV',
'dif_porc_t_1_saldo_tot_TC_X_SMLV',
'dif_porc_t_6_saldo_tot_TC_X_SMLV',
'dif_porc_t_1_saldo_tot_TC_T_SMLV',
'dif_porc_t_6_saldo_tot_TC_T_SMLV',
'dif_porc_t_1_saldo_tot_tot_SMLV',
'dif_porc_t_6_saldo_tot_tot_SMLV',
'dif_porc_t_1_cuota_tot_LB_SMLV',
'dif_porc_t_6_cuota_tot_LB_SMLV',
'dif_porc_t_1_cuota_tot_TC_SMLV',
'dif_porc_t_6_cuota_tot_TC_SMLV',
'dif_porc_t_1_cuota_tot_NV_SMLV',
'dif_porc_t_6_cuota_tot_NV_SMLV',
'dif_porc_t_1_cuota_tot_CE_SMLV',
'dif_porc_t_6_cuota_tot_CE_SMLV',
'dif_porc_t_1_cuota_tot_LE_SMLV',
'dif_porc_t_6_cuota_tot_LE_SMLV',
'dif_porc_t_1_cuota_tot_TC_X_SMLV',
'dif_porc_t_6_cuota_tot_TC_X_SMLV',
'dif_porc_t_1_cuota_tot_TC_T_SMLV',
'dif_porc_t_6_cuota_tot_TC_T_SMLV',
'dif_porc_t_1_cuota_tot_tot_SMLV',
'dif_porc_t_6_cuota_tot_tot_SMLV',
'dif_porc_t_1_cupo_tot_LB_SMLV',
'dif_porc_t_6_cupo_tot_LB_SMLV',
'dif_porc_t_1_cupo_tot_TC_SMLV',
'dif_porc_t_6_cupo_tot_TC_SMLV',
'dif_porc_t_1_cupo_tot_NV_SMLV',
'dif_porc_t_6_cupo_tot_NV_SMLV',
'dif_porc_t_1_cupo_tot_SO_SMLV',
'dif_porc_t_6_cupo_tot_SO_SMLV',
'dif_porc_t_1_cupo_tot_TC_X_SMLV',
'dif_porc_t_6_cupo_tot_TC_X_SMLV',
'dif_porc_t_1_cupo_tot_TC_T_SMLV',
'dif_porc_t_6_cupo_tot_TC_T_SMLV',
'dif_porc_t_1_cupo_tot_tot_SMLV',
'dif_porc_t_6_cupo_tot_tot_SMLV',
'dif_porc_t_1_utilizacion_prom_TC_X',
'dif_porc_t_6_utilizacion_prom_TC_X',
'dif_porc_t_1_utilizacion_prom_TC_T',
'dif_porc_t_6_utilizacion_prom_TC_T',
'dif_porc_t_1_utilizacion_prom_TC',
'dif_porc_t_6_utilizacion_prom_TC',
'dif_porc_t_1_amortizacion_prom_LB',
'dif_porc_t_6_amortizacion_prom_LB',
'dif_porc_t_1_amortizacion_prom_NV',
'dif_porc_t_6_amortizacion_prom_NV',
'dif_porc_t_1_amortizacion_prom_CE',
'dif_porc_t_1_amortizacion_prom_LE',
'dif_porc_t_6_amortizacion_prom_LE',
'dif_porc_t_1_amortizacion_prom_tot',
'dif_porc_t_6_amortizacion_prom_tot',
'dif_porc_t_1_porc_obl_mora1_30_TC',
'dif_porc_t_6_porc_obl_mora1_30_TC',
'dif_porc_t_1_porc_obl_mora1_30_NV',
'dif_porc_t_6_porc_obl_mora1_30_NV',
'dif_porc_t_1_porc_obl_mora31_60_NV',
'dif_porc_t_6_porc_obl_mora31_60_NV',
'dif_porc_t_1_porc_obl_mora1_30_LE',
'dif_porc_t_1_porc_obl_mora1_30_TC_X',
'dif_porc_t_6_porc_obl_mora1_30_TC_X',
'dif_porc_t_1_porc_obl_mora1_30_tot',
'dif_porc_t_6_porc_obl_mora1_30_tot',
'dif_porc_t_1_porc_obl_mora31_60_tot',
'dif_porc_t_6_porc_obl_mora31_60_tot',
'cupo_prom_ult3_meses_total',
'cuota_paga_prom_ult7_meses_total',
'cupo_prom_ult7_meses_total',
'mora_max_ult3_meses_total',
'saldo_prom_ult3_meses_total',
'saldo_prom_ult6_meses_total',
'util_prom_ult6_meses_total',
'mora_cierre_ult_mes_total',
'mora_max_ult6_meses_total',
'util_prom_ult9_meses_total',
'saldo_prom_ult7_meses_total',
'util_prom_ult3_meses_total',
'cuota_paga_prom_ult3_meses_total',
'util_prom_ult9_meses_expr',
'util_prom_ult6_meses_expr',
'mora_max_ult3_meses_expr',
'saldo_prom_ult3_meses_expr',
'mora_max_ult6_meses_expr',
'saldo_prom_ult6_meses_expr',
'util_prom_ult3_meses_expr',
'mora_cierre_ult_mes_expr',
'cupo_prom_ult7_meses_expr',
'cupo_prom_ult3_meses_expr',
'saldo_prom_ult7_meses_expr',
'saldo_prom_ult3_meses_trad',
'util_prom_ult9_meses_trad',
'mora_max_ult6_meses_trad',
'cupo_prom_ult7_meses_trad',
'util_prom_ult6_meses_trad',
'saldo_prom_ult6_meses_trad',
'util_prom_ult3_meses_trad',
'cupo_prom_ult3_meses_trad',
'saldo_prom_ult7_meses_trad',
'num_lib_solicitadas',
'prom_monto_solicitado',
'prom_monto_aprobado',
'prom_monto_desembolsado',
'prom_monto_novado',
'prom_prop_desembolso',
'prom_n_cuotas',
'prom_valor_cuota',
'prom_tasa',
'prom_dias_procesamiento',
'max_dias_procesamiento',
'n_novaciones',
'pasv_num_ctas_vig_tot',
'pasv_num_ctas_vig_cc',
'pasv_num_ctas_vig_ca',
'pasv_num_ctas_vig_cdt',
'pasv_dias_desde_ultima_trans',
'pasv_antig_total',
'pasv_num_meses_ult_apertura',
'pasv_saldo_tot_fin_mes_smlv',
'pasv_saldo_cc_fin_mes_smlv',
'pasv_saldo_ca_fin_mes_smlv',
'pasv_saldo_cdt_fin_mes_smlv',
'pasv_saldo_min_fin_mes_smlv',
'pasv_saldo_min_cc_fin_mes_smlv',
'pasv_saldo_min_ca_fin_mes_smlv',
'pasv_saldo_min_cdt_fin_mes_smlv',
'pasv_saldo_max_fin_mes_smlv',
'pasv_saldo_max_cc_fin_mes_smlv',
'pasv_saldo_max_ca_fin_mes_smlv',
'pasv_saldo_max_cdt_fin_mes_smlv',
'pasv_saldo_total_1mes_atras',
'pasv_saldo_cc_1mes_atras',
'pasv_saldo_ca_1mes_atras',
'pasv_saldo_cdt_1mes_atras',
'pasv_saldo_total_3mes_atras',
'pasv_saldo_cc_3mes_atras',
'pasv_saldo_ca_3mes_atras',
'pasv_saldo_cdt_3mes_atras',
'pasv_saldo_total_6mes_atras',
'pasv_saldo_cc_6mes_atras',
'pasv_saldo_ca_6mes_atras',
'pasv_saldo_cdt_6mes_atras',
'fecha_ult_tx',
'dias_desde_ult_tx',
'num_tx_mes_ant',
'prom_mov_mes_ant',
'fecha_ult_tx_expr',
'dias_desde_ult_tx_expr',
'num_tx_mes_ant_expr',
'prom_mov_mes_ant_expr',
'fecha_ult_tx_trad',
'dias_desde_ult_tx_trad',
'num_tx_mes_ant_trad',
'prom_mov_mes_ant_trad',
#'fecha_ult_tx_la14',
'num_tx_mes_ant_la14',
'prom_mov_mes_ant_la14',
'num_tx_ult_1mes',
'num_tx_ult_2meses',
'num_tx_ult_3meses',
'num_tx_ult_4meses',
'num_tx_ult_6meses',
'prom_tx_ult_1mes',
'prom_tx_ult_2meses',
'prom_tx_ult_3meses',
'prom_tx_ult_4meses',
'prom_tx_ult_6meses',
'valor_mov_ult_1mes',
'valor_mov_ult_2meses',
'valor_mov_ult_3meses',
'valor_mov_ult_4meses',
'valor_mov_ult_6meses',
'prom_mov_ult_1mes',
'prom_mov_ult_2meses',
'prom_mov_ult_3meses',
'prom_mov_ult_4meses',
'prom_mov_ult_6meses',
'prom_cuot_ult_1mes',
'prom_cuot_ult_2meses',
'prom_cuot_ult_3meses',
'prom_cuot_ult_4meses',
'prom_cuot_ult_6meses',
'num_tx_ult_1mes_trad',
'num_tx_ult_2meses_trad',
'num_tx_ult_3meses_trad',
'num_tx_ult_4meses_trad',
'num_tx_ult_6meses_trad',
'prom_tx_ult_1mes_trad',
'prom_tx_ult_2meses_trad',
'prom_tx_ult_3meses_trad',
'prom_tx_ult_4meses_trad',
'prom_tx_ult_6meses_trad',
'valor_mov_ult_1mes_trad',
'valor_mov_ult_2meses_trad',
'valor_mov_ult_3meses_trad',
'valor_mov_ult_4meses_trad',
'valor_mov_ult_6meses_trad',
'prom_mov_ult_1mes_trad',
'prom_mov_ult_2meses_trad',
'prom_mov_ult_3meses_trad',
'prom_mov_ult_4meses_trad',
'prom_mov_ult_6meses_trad',
'num_tx_ult_1mes_expr',
'num_tx_ult_2meses_expr',
'num_tx_ult_3meses_expr',
'num_tx_ult_4meses_expr',
'num_tx_ult_6meses_expr',
'prom_tx_ult_1mes_expr',
'prom_tx_ult_2meses_expr',
'prom_tx_ult_3meses_expr',
'prom_tx_ult_4meses_expr',
'prom_tx_ult_6meses_expr',
'valor_mov_ult_1mes_expr',
'valor_mov_ult_2meses_expr',
'valor_mov_ult_3meses_expr',
'valor_mov_ult_4meses_expr',
'valor_mov_ult_6meses_expr',
'prom_mov_ult_1mes_expr',
'prom_mov_ult_2meses_expr',
'prom_mov_ult_3meses_expr',
'prom_mov_ult_4meses_expr',
'prom_mov_ult_6meses_expr',
'prom_cuot_ult_1mes_expr',
'prom_cuot_ult_2meses_expr',
'prom_cuot_ult_3meses_expr',
'prom_cuot_ult_4meses_expr',
'prom_cuot_ult_6meses_expr',
'num_tx_ult_1mes_14',
'num_tx_ult_2meses_14',
'num_tx_ult_3meses_14',
'num_tx_ult_4meses_14',
'num_tx_ult_6meses_14',
'prom_tx_ult_1mes_14',
'prom_tx_ult_2meses_14',
'prom_tx_ult_3meses_14',
'prom_tx_ult_4meses_14',
'prom_tx_ult_6meses_14',
'valor_mov_ult_1mes_14',
'valor_mov_ult_2meses_14',
'valor_mov_ult_3meses_14',
'valor_mov_ult_4meses_14',
'valor_mov_ult_6meses_14',
'prom_mov_ult_1mes_14',
'prom_mov_ult_2meses_14',
'prom_mov_ult_3meses_14',
'prom_mov_ult_4meses_14',
'prom_mov_ult_6meses_14',
'num_tx_ult_1mes_pasv',
'num_tx_ult_2meses_pasv',
'num_tx_ult_3meses_pasv',
'num_tx_ult_4meses_pasv',
'num_tx_ult_6meses_pasv',
'prom_tx_ult_1meses_pasv',
'prom_tx_ult_2meses_pasv',
'prom_tx_ult_3meses_pasv',
'prom_tx_ult_4meses_pasv',
'prom_tx_ult_6meses_pasv',
'sum_tx_linea_pasv',
'sum_tx_nolinea_pasv',
'dias_desde_ult_tx_pasv',
'num_tx_ult_1mes_cte',
'num_tx_ult_2meses_cte',
'num_tx_ult_3meses_cte',
'num_tx_ult_4meses_cte',
'num_tx_ult_6meses_cte',
'prom_tx_ult_1meses_cte',
'prom_tx_ult_2meses_cte',
'prom_tx_ult_3meses_cte',
'prom_tx_ult_4meses_cte',
'prom_tx_ult_6meses_cte',
'sum_tx_linea_cte',
'sum_tx_nolinea_cte',
'dias_desde_ult_tx_cte',
'num_tx_ult_1mes_cdt',
'num_tx_ult_2meses_cdt',
'num_tx_ult_3meses_cdt',
'num_tx_ult_4meses_cdt',
'num_tx_ult_6meses_cdt',
'prom_tx_ult_1meses_cdt',
'prom_tx_ult_2meses_cdt',
'prom_tx_ult_3meses_cdt',
'prom_tx_ult_4meses_cdt',
'prom_tx_ult_6meses_cdt',
'sum_tx_linea_cdt',
'sum_tx_nolinea_cdt',
'dias_desde_ult_tx_cdt',
'num_tx_ult_1mes_ahr',
'num_tx_ult_2meses_ahr',
'num_tx_ult_3meses_ahr',
'num_tx_ult_4meses_ahr',
'num_tx_ult_6meses_ahr',
'prom_tx_ult_1meses_ahr',
'prom_tx_ult_2meses_ahr',
'prom_tx_ult_3meses_ahr',
'prom_tx_ult_4meses_ahr',
'prom_tx_ult_6meses_ahr',
'sum_tx_linea_ahr',
'sum_tx_nolinea_ahr',
'dias_desde_ult_tx_ahr'
]


In [69]:
#Función que calcula el indice de estabilidad poblacional PSI para las variables continuas
def psi_continuas(variable,base,var_fecha):
    base['decil_'+variable]=pd.qcut(base[variable],5,duplicates='drop')
    tabla_5=base.groupby(['decil_'+variable])['decil_'+variable].count().reset_index(name='conteo')
    variable_total=tabla_5['conteo'].sum()
    tabla_5['dist']=tabla_5['conteo']/variable_total

    tabla_6=base.groupby(['decil_'+variable,var_fecha])['decil_'+variable].count().reset_index(name='conteo')
    tabla_7=base.groupby([var_fecha])['decil_'+variable].count().reset_index(name='total')

    tabla_8=tabla_6.merge(tabla_7,how='left',on=[var_fecha])
    tabla_8['dist_mes']=tabla_8['conteo']/tabla_8['total']

    tabla_9=tabla_8.merge(tabla_5,how='left',on=['decil_'+variable]).sort_values(by=[var_fecha]).reset_index()
    tabla_9['PSI']=(tabla_9['dist_mes']-tabla_9['dist'])*np.log(tabla_9['dist_mes']/tabla_9['dist'])
    tabla_10=tabla_9.groupby([var_fecha])['PSI'].sum().reset_index(name=variable)

    
    return pd.DataFrame([[variable,tabla_10.max()[1].astype("float64")]])


In [71]:
var

'ult_aÃ±o_moras_120_cartera_hist'

In [72]:
master_table['ult_año_moras_120_cartera_hist']

0        0.0
1        0.0
2        NaN
3        0.0
4        NaN
        ... 
35609    0.0
35610    NaN
35611    0.0
35612    NaN
35613    0.0
Name: ult_año_moras_120_cartera_hist, Length: 35614, dtype: float64

In [74]:
estab_con=pd.DataFrame()
for var in variables_continuas_pre:
    a=psi_continuas(var,master_table,'fecha_psi')
    estab_con=estab_con.append(a)

In [29]:
# # quitar 'fecha_ult_tx_la14'
# estab_con=pd.DataFrame()
# for var in variables_continuas_perf:
#     a=psi_continuas(var,master_table,'fecha_psi')
#     estab_con=estab_con.append(a)

In [75]:
#Función que calcula el indice de estabilidad poblacional PSI para las variables categoricas
def psi_categorica(variable,base,var_fecha): 
    tabla_5=base.groupby([variable])[variable].count().reset_index(name='conteo')
    variable_total=tabla_5['conteo'].sum()
    tabla_5['dist']=tabla_5['conteo']/variable_total

    tabla_6=base.groupby([variable,var_fecha])[variable].count().reset_index(name='conteo')
    tabla_7=base.groupby([var_fecha])[variable].count().reset_index(name='total')

    tabla_8=tabla_6.merge(tabla_7,how='left',on=[var_fecha])
    tabla_8['dist_mes']=tabla_8['conteo']/tabla_8['total']
    tabla_9=tabla_8.merge(tabla_5,how='left',on=[variable]).sort_values(by=[var_fecha]).reset_index()
    tabla_9['PSI']=(tabla_9['dist_mes']-tabla_9['dist'])*np.log(tabla_9['dist_mes']/tabla_9['dist'])
    tabla_10=tabla_9.groupby([var_fecha])['PSI'].sum().reset_index(name=variable)
    return pd.DataFrame([[variable,tabla_10.max()[1]]])

In [76]:
#list(master_table.dtypes[master_table.dtypes=='object'].index)

['tipo_campana',
 'periodo',
 'cedulaenc',
 'ciudad_de_expedicion',
 'genero',
 'peor_califi_trim_1_endeud',
 'peor_calif_trim_2_endeud',
 'rango_aproximado_edad',
 'tipo_cliente',
 'tipo_id',
 'derogatorio',
 'marca_derogatorio',
 'tdc_altura_maxima_de_mora_was_is',
 'cartera_banca_alt_max_de_mora_was_is',
 'cartera_coope_alt_max_de_mora_was_is',
 'cartera_hipote_alt_max_de_mora_was_is',
 'moramax_mercado_was_is',
 'was_is_LB',
 'was_is_TC',
 'was_is_NV',
 'was_is_CH',
 'was_is_CE',
 'was_is_LE',
 'was_is_ME',
 'was_is_SO',
 'was_is_CC',
 'was_is_TC_X',
 'was_is_TC_T',
 'was_is_tot',
 'sector_ultlibranza',
 'subsector_ultlibranza',
 'tipo_pagaduria_ultlibranza',
 'mode_dia',
 'mode_descripcion_tx',
 'mode_pais_tx',
 'mode_dia_expr',
 'mode_descripcion_expr_tx_expr',
 'mode_dia_trad',
 'mode_descripcion_trad_tx_trad',
 'mode_pais_tx_trad',
 'mode_dia_la14',
 'mode_descripcion_la14_tx_la14',
 'mode_dia_pasv',
 'estado_cta_actual_pasv',
 'tipo_transaccion',
 'fecha_ult_tx_pasv',
 'mode_d

In [80]:
#listado de variables categoricas
variables_categoricas=[
 'ciudad_de_expedicion',
 'genero',
 'peor_califi_trim_1_endeud',
 'peor_calif_trim_2_endeud',
 'rango_aproximado_edad',
 'tipo_cliente',
 'tipo_id',
 'derogatorio',
 'marca_derogatorio',
 'tdc_altura_maxima_de_mora_was_is',
 'cartera_banca_alt_max_de_mora_was_is',
 'cartera_coope_alt_max_de_mora_was_is',
 'cartera_hipote_alt_max_de_mora_was_is',
 'moramax_mercado_was_is',
 'was_is_LB',
 'was_is_TC',
 'was_is_NV',
 'was_is_CH',
 'was_is_CE',
 'was_is_LE',
 'was_is_ME',
 'was_is_SO',
 'was_is_CC',
 'was_is_TC_X',
 'was_is_TC_T',
 'was_is_tot',
 'sector_ultlibranza',
 'subsector_ultlibranza',
 'tipo_pagaduria_ultlibranza',
 'mode_dia',
 'mode_descripcion_tx',
 'mode_pais_tx',
 'mode_dia_expr',
 'mode_descripcion_expr_tx_expr',
 'mode_dia_trad',
 'mode_descripcion_trad_tx_trad',
 'mode_pais_tx_trad',
 'mode_dia_la14',
 'mode_descripcion_la14_tx_la14',
 'mode_dia_pasv',
 'estado_cta_actual_pasv',
 'tipo_transaccion',
 'fecha_ult_tx_pasv',
 'mode_dia_cte',
 'estado_cta_actual_cte',
 'tipo_transaccion_cte',
 'fecha_ult_tx_cte',
 'mode_dia_cdt',
 'estado_cta_actual_cdt',
 'tipo_transaccion_cdt',
 'fecha_ult_tx_cdt',
 'mode_dia_ahr',
 'estado_cta_actual_ahr',
 'tipo_transaccion_ahr',
 'fecha_ult_tx_ahr'
]

In [81]:
#Se ejecuta la funcion para todas las variables categoricas
estab_cat=pd.DataFrame()
for var in variables_categoricas:
    estab_cat=pd.concat([estab_cat,psi_categorica(var,master_table,'fecha_psi')])

In [82]:
#Se unen los psi de categoricas y continuas
estabilidad=estab_cat.append(estab_con)

In [83]:
estabilidad["index"]=estabilidad.loc[:,0]
estabilidad["PSI_max"]=estabilidad.loc[:,1]

### 4. Decision Final

In [84]:
# Si la variable tiene mas del x% de missing o no es variable o no es estable se elimina de la base

In [85]:
c_v=completitud.merge(variabilidad,how='left',on='index')

In [86]:
c_v_e=c_v.merge(estabilidad,how='left',on='index')

In [87]:
c_v_e['decision_final']=np.where((c_v_e['variabilidad']=='No variable') | (c_v_e['Porcentaje_Missing']>0.95) | (c_v_e['PSI_max']>0.25),'Excluir','Incluir')

In [88]:
c_v_e

,index,Porcentaje_Missing,Percentil10,Percentil90,variabilidad,0,1,PSI_max,decision_final
0,id_cliente,0.000000,1.915050e+17,9.115247e+17,Variable,NaN,NaN,NaN,Incluir
1,fecha_tx_pav,0.000000,2.019020e+05,2.019100e+05,Variable,NaN,NaN,NaN,Incluir
2,fecha_tx_act,0.000000,2.019020e+05,2.019100e+05,Variable,NaN,NaN,NaN,Incluir
3,fecha_pasivo,0.000000,2.019010e+05,2.019090e+05,Variable,NaN,NaN,NaN,Incluir
4,fecha_lib,0.000000,2.019020e+05,2.019100e+05,Variable,NaN,NaN,NaN,Incluir
...,...,...,...,...,...,...,...,...,...
1555,sum_tx_linea_ahr,0.517156,1.000000e+00,1.800000e+01,Variable,sum_tx_linea_ahr,0.095449,0.095449,Incluir
1556,sum_tx_nolinea_ahr,0.432162,1.000000e+00,3.200000e+01,Variable,sum_tx_nolinea_ahr,1.555418,1.555418,Excluir
1557,fecha_ult_tx_ahr,0.406273,NaN,NaN,NaN,fecha_ult_tx_ahr,3.068917,3.068917,Excluir
1558,dias_desde_ult_tx_ahr,0.406273,0.000000e+00,1.300000e+01,Variable,dias_desde_ult_tx_ahr,0.121320,0.121320,Incluir


In [89]:
c_v_e.to_csv("/mnt/work/CU_adquisicionTC/Notebooks/creacion_modelo/compl_var_est_Seg_PAPROB.csv")

In [41]:
# c_v_e.to_csv("/mnt/work/CU_adquisicionTC/Notebooks/creacion_modelo/compl_var_est_Seg_PERF.csv")